In [15]:
import pandas as pd
import re
import html.parser
from sklearn.feature_extraction.text import CountVectorizer
import pickle
from nltk.corpus import stopwords
from nltk.stem import porter
import nltk
import pickle
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation

stemmer = porter.PorterStemmer()

stopwords = stopwords.words()

In [5]:
f = open('mongodatabase.pkl','rb')
df = pickle.load(f)

In [3]:
added_stopwords = ['wait','cant','bi','hi','always','long','short','getting','away','see','watching','one','better','today','get','th',
'could','feel','would','hours','must','watch','st','theres','much','probably','came','brought','lot','last','bit',
'youre','need','make','piece','ass','nobody','thought','cosa','think','right','behind','late','bringing','facupfinal',
'cup','troian','half','find','really','many','never','oh','actual','ser','ways','stream','via','name','street','vote',
'something','nothing','still','top','set','didnt','us','along','two','lots','everyone','shes','turn','well','high',
'anyone','says','become','ten','mo','gets','click','gave','give','knew','forward','ver','voy','definitely','haha',
'sat','join','old','fucking','hrh','ms','sure','gonna','whatever','making','since','price', 'got','may','let','litte',
'aqu','without','year old','big','sometimes','nd','look','got']

In [4]:
stopwords = stopwords + added_stopwords

In [8]:
df.tail(5)

,created_at,location,screen_name,source,text
899929,Tue May 22 02:21:10 +0000 2018,"Oklahoma City, OK",MICHAEL44LXCCC,"<a href=""http://twitter.com/download/android"" ...","okaY lOOk, wE alL nO thE sayinG, ""yoU arE waT ..."
899930,Tue May 22 02:21:11 +0000 2018,ONtheRiNGSofSATURN,MAMA2a_PRiNCE,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",:) https://t.co/lSgDZdxWP3
899931,Tue May 22 02:21:13 +0000 2018,"Los Angeles, CA",WDdresses,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",7 White Dresses to Shop for Summer Inspired by...
899932,Tue May 22 02:21:13 +0000 2018,Mono-coastal; bi-city,salys,"<a href=""https://paper.li"" rel=""nofollow"">Pape...",The latest The news-politix Daily! https://t.c...
899933,Tue May 22 02:21:14 +0000 2018,StyleLand,PinkGlamChic,"<a href=""https://paper.li"" rel=""nofollow"">Pape...",The latest https://t.co/KMPOIGo0Fg! https://t....


In [28]:
def one_tweet_clean(tweet):
    punctuations = '''!()-[]{};:'"\,<>./?$%^&*_~'''
    new_doc = re.sub(r'http\S+', '', tweet)
    new_doc = html.unescape(new_doc)
    new_doc = new_doc.replace("\n", "")
    new_doc = new_doc.replace("\\","")
    new_doc = new_doc.replace("@Suits_USA","Suits_USA")
    new_doc = new_doc.replace("@RoyalFamily","RoyalFamily")
    new_doc = new_doc.replace("@BishopCurry","BishopCurry")
    new_doc = new_doc.replace("@KensingtonRoyal","KensingtonRoyal")
    new_doc = new_doc.lower()
    new_doc = re.sub(r'@\w+','',new_doc)
    new_doc = re.sub(r'[0-9]', '',new_doc)
    new_doc = stemmer.stem(new_doc)
    for punc in punctuations:
        new_doc = new_doc.replace(punc, "")
    if new_doc in stopwords:
        return None
    return new_doc

In [31]:
df['cleaned_text'] = df['text'].apply(lambda x: one_tweet_clean(x))

In [33]:
df.to_pickle('updateddataframe.pkl')

In [32]:
df.head(5)

,created_at,location,screen_name,source,text,cleaned_text
0,Fri May 18 23:32:39 +0000 2018,Midwest USA - Blog:,SueSpenceDaniel,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Royal Wedding countdown... can't wait! #RoyalW...,royal wedding countdown cant wait #royalwed
1,Fri May 18 23:32:40 +0000 2018,None,AsliAyan1,"<a href=""http://twitter.com/download/iphone"" r...",İngiliz kraliyetini yıllardır çok severek taki...,i̇ngiliz kraliyetini yıllardır çok severek tak...
2,Fri May 18 23:32:41 +0000 2018,"Nairobi, Kenya",VictorMochere,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",DONATE:\n\nAs a football fan and an adventurer...,donateas a football fan and an adventurer it h...
3,Fri May 18 23:32:41 +0000 2018,Curitiba,Nath_Maia,"<a href=""http://twitter.com/download/android"" ...",Só queria dizer que se eu casasse com o Princi...,só queria dizer que se eu casasse com o princi...
4,Fri May 18 23:32:44 +0000 2018,Lanarkshire Sco,weemazsmith,"<a href=""http://twitter.com/download/android"" ...",What am I going to do ! My fav prince is getti...,what am i going to do my fav prince is gettin...


In [30]:
def clean_first(text):
    new_text = []
    punctuations = '''!()-[]{};:'"\,<>./?$%^&*_~'''
    for doc in text:  
        new_doc = one_tweet_clean(doc)
        if doc is not None:   
            new_text.append(new_doc)
    return new_text

In [7]:
docs = df['text'].values.tolist()

In [8]:
new_docs = clean_first(docs)

In [11]:
with open('cleandata.pkl','wb') as file:
    pickle.dump(new_docs, file)

In [7]:
with open('cleandata.pkl','rb') as file:
    new_docs = pickle.load(file)

In [8]:
# takes out Chinese, Japanese, and Korean characters with unicode data
def emoticon_tokenize(s):
    token_pattern = r'(?<=^|(?<=\b))[\w\#]\w+\b'
    emoji_regex = re.compile(r'[\u263a-\U0001f645]')
    token_regex = re.compile(token_pattern, flags=re.ASCII)
    emoticons = emoji_regex.findall(s)
    parsed_emoticons=[] 
    for e in emoticons: 
        try: 
            if unicodedata.name(e)[:3]!='CJK' and unicodedata.name(e)[:8]!='HIRAGANA' and unicodedata.name(e)[:8]!='KATAKANA':
                parsed_emoticons.append(e)
        except: 
            pass
    word_list = token_regex.findall(s)
    return word_list + parsed_emoticons

In [9]:
%%time
cv = CountVectorizer(ngram_range=(1, 3), tokenizer = emoticon_tokenize, max_features=3000, stop_words=stopwords, max_df = 0.6)
cv.fit(new_docs)
len(cv.vocabulary_)

CPU times: user 51.8 s, sys: 2.76 s, total: 54.5 s
Wall time: 56.6 s


In [23]:
with open('cv.pkl','wb') as file:
    pickle.dump(cv, file)

In [10]:
cv.vocabulary_

{'royal': 2216,
 'wedding': 2848,
 'royalwed': 2240,
 'royal wedding': 2222,
 'ok': 1851,
 'lady': 1434,
 'diana': 712,
 '#royalwedding': 37,
 'harryandmeghan': 1188,
 'lady diana': 1435,
 '#royalwedding harryandmeghan': 39,
 'football': 1003,
 'fan': 929,
 'dream': 753,
 'attend': 187,
 'chance': 478,
 'help': 1218,
 'raise': 2119,
 'know': 1423,
 'shot': 2412,
 'worth': 2957,
 'try': 2717,
 'thank': 2627,
 '#royalwed': 35,
 'queria': 2099,
 'principe': 2053,
 'inglaterra': 1307,
 'show': 2415,
 'girls': 1070,
 'casamento': 436,
 'going': 1094,
 'fav': 941,
 'prince': 2017,
 'married': 1619,
 'night': 1816,
 'royalwedding': 2244,
 'princeharry': 2033,
 'royalwedding princeharry': 2290,
 'gran': 1118,
 'boda': 324,
 'real': 2133,
 'boda real': 327,
 'yes': 2980,
 'tiara': 2656,
 'wears': 2844,
 'guide': 1136,
 'great': 1123,
 'funeral': 1032,
 'princess': 2041,
 'duke': 777,
 'duchess': 770,
 'cambridge': 412,
 'princess diana': 2043,
 'duke duchess': 778,
 'monarchy': 1744,
 'changing

In [11]:
cv_data = cv.transform(new_docs)

In [12]:
def display_topics(model, feature_names, num_top_words, topic_names=None):  
    for ix, topic in enumerate(model.components_):  #model.components_ are topic probability distributions
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-num_top_words - 1:-1]]))

In [16]:
n_comp = 5   #number of topics
lsa_cv = TruncatedSVD(n_components=n_comp)
nmf_cv = NMF(n_components=n_comp)

In [19]:
# Try the LDA model as well
lda_cv = LatentDirichletAllocation(n_components=n_comp)

In [17]:
%%time
#trained models. so lsa_cv is already fitted and transformed. Think of lr.fit() --> lr.coef_
# going from big document space to feature space. Think about rabbit and dish

lsa_cv_data = lsa_cv.fit_transform(cv_data)

CPU times: user 3.42 s, sys: 754 ms, total: 4.17 s
Wall time: 3.66 s


In [18]:
nmf_cv_data = nmf_cv.fit_transform(cv_data)

In [21]:
with open('nmf_cv.pkl','wb') as file:
    pickle.dump(nmf_cv, file)

In [22]:
with open('lsa_cv.pkl','wb') as file:
    pickle.dump(lsa_cv, file)

In [26]:
%%time
lda_cv_data = lda_cv.fit_transform(cv_data)

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


CPU times: user 11min 40s, sys: 4.49 s, total: 11min 45s
Wall time: 11min 55s


In [19]:
# display top 5 features of the lsa model
display_topics(lsa_cv,cv.get_feature_names(),30)   #10 is the number of top words


Topic  0
royalwedding, wedding, meghan, harry, royal, royalwed, prince, harryandmeghan, love, royal wedding, prince harry, markle, day, harry meghan, meghan markle, royalwedding royalwedding, royalwedding harryandmeghan, like, beautiful, meghanmarkle, prince harry meghan, princeharry, windsor, latest, harry meghan markle, #royalwedding, dress, people, sussex, happy

Topic  1
royalwedding, royalwedding royalwedding, royalwedding harryandmeghan, thanks royalwedding, daily thanks royalwedding, harryandmeghan royalwedding, daily royalwedding, royalwedding royalwedding harryandmeghan, hiring careers, jobs hiring careers, jobs hiring, hiring, careers, royalwedding coverage, #facupfinal, ready royalwedding, jobs, #royalwedding royalwedding, excited royalwedding, blockchain, sunday royalwedding, love royalwedding, beautiful royalwedding, cryptocurrency blockchain job, blockchain job, cryptocurrency, royalwedding meghanmarkle, cryptocurrency blockchain, coverage royalwedding, sunday

Topic  2


In [20]:
# display top 5 features of the nmf model
display_topics(nmf_cv,cv.get_feature_names(),30)


Topic  0
royalwedding, harryandmeghan, royalwedding royalwedding, royalwedding harryandmeghan, day, love, latest, like, thanks, meghanmarkle, beautiful, princeharry, daily, thanks royalwedding, windsor, people, time, dress, best, happy, good, live, news, know, harryandmeghan royalwedding, world, queen, morning, princess, going

Topic  1
royalwed, love, like, looks, dress, latest, beautiful, meghan, beckham, people, thanks, harry, daily, queen, clooney, amal, wedding royalwed, black, know, time, going, way, royalwedding royalwed, george, looking, david, victoria, suits, good, kate

Topic  2
meghan, harry, prince, prince harry, markle, harry meghan, meghan markle, prince harry meghan, harry meghan markle, sussex, duchess, congratulations, duke, duchess sussex, day, duke duchess, windsor, duke duchess sussex, married, meghan harry, wedding, markles, meghan markles, markle royalwedding, pr, congratulations prince, meghan markle royalwedding, congratulations prince harry, ncipe, pr ncipe



In [27]:
# display top 5 topics for the lda model
display_topics(lda_cv, cv.get_feature_names(),30)


Topic  0
#royalwedding, meghan, harry, royalwedding, prince, markle, prince harry, meghan markle, harry meghan, meghanmarkle, windsor, prince harry meghan, royalfamily, stunning, harry meghan markle, official, photos, princeharry, white, twitter, suits, wedding, proud, kensingtonroayl, rt, foto, william, choir, king, trump

Topic  1
royalwed, royalwedding, dress, world, real, made, first, wedding, george, cute, kate, back, pr, boda, morning, live, actually, bbmas, ceremony, thats, ncipe, pr ncipe, diana, lady, read, house, charlotte, pretty, believe, mom

Topic  2
royalwedding, wedding, love, royal, harryandmeghan, beautiful, royal wedding, day, best, time, duchess, royalwedding harryandmeghan, sussex, life, great, princeharry, happy, couple, duke, duchess sussex, meghanmarkle, royalwedding royalwedding, new, duke duchess, congratulations, moment, lovely, please, omg, yes

Topic  3
royalwedding, royalwed, latest, thanks, daily, way, thanks royalwedding, #royalwed, news, looking, daily